# Magicbricks scraper
Built a web-scraping algorithm that scrapes from a dynamic website involving a lot more
complexity and variables

- **The algorithm needs to `open each listing page and copy data from those pages`**
- **The algorithm needs to take care of `infinite scrolling on the website.`**
- **Captured approx 500 listings**

### To scrape
- Headline
- Price
- Address
- Bedrooms
- Bathrooms
- Balcony
- Super Area
- Price per sqft
- Status
- Transaction Type
- Floor
- Car parking
- Furnished status
- Lifts
- Description
- Price breakup
- Address
- Landmarks
- Age of construction
- Price comparison
- Expected rent
- Monthly EMI
- Owner/Builder

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

### First scraping properties from main page

<img src='p_list.png' />

In [106]:
url="https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa,Residential-Plot&cityName=Bangalore"
header={"user-agent":"Mozilla/4.0 (compatible; MSIE 6.0; Windows 98) XX"}
soup=BeautifulSoup(requests.get(url,headers=header).text)

This website has saved its product url inside `div` tag with class_=`m-srp-card SRCard` inside `data-code` attribute with some other details, so extracting just the url using regex

In [107]:
property_links=[]
for i in soup.find_all('div',class_="m-srp-card SRCard"):
    property_links.append(re.findall("https://www.magicbricks.com/.+",i.get("data-code"))[0].split(",")[0].strip("'"))
len(property_links)

29

-------

### Dealing with infinite scrolling
So **`Ajax`** enables websites to use infinite scrolling. But that ajax request also has a URL from where the products are loading on the same page on scroll.
- Open the page in Google Chrome
- Then go to console ; right click and enable LogXMLHttpRequests.
- Now reload the page and scroll slowly. When new products are populated, you will see different URLs named after “XHR finished loading: GET”. Click on them.

In [118]:
#generating url pages from a template url
num_pages=40
index=np.array(range(num_pages))*30
urls=[f"https://www.magicbricks.com/mbsearch/propertySearch.html?propertyType_new=10002_10003_10021_10022,10001_10017,10000&postedSince=-1&city=3327&searchType=1&propertyType=10002,10003,10021,10022,10001,10017,10000&disWeb=Y&pType=10002,10003,10021,10022,10001,10017,10000&category=S&cusImgCount=0&groupstart={page}&offset=0&maxOffset=431&attractiveIds=&page=2&ltrIds=46765693,49757250,48496323,49399225,49102575,49830142,39475611,46950205,49168149,49454097,45478005,43585785,49472179,47257529,48775091,49211061,35809193,49031201,48434303,49306755,49098265,49573236,46367371,35508545,50293354,39824397,49637258,36526459&preCompiledProp=&excludePropIds=&addpropertyDataSet="
      for page in index]
urls

['https://www.magicbricks.com/mbsearch/propertySearch.html?propertyType_new=10002_10003_10021_10022,10001_10017,10000&postedSince=-1&city=3327&searchType=1&propertyType=10002,10003,10021,10022,10001,10017,10000&disWeb=Y&pType=10002,10003,10021,10022,10001,10017,10000&category=S&cusImgCount=0&groupstart=0&offset=0&maxOffset=431&attractiveIds=&page=2&ltrIds=46765693,49757250,48496323,49399225,49102575,49830142,39475611,46950205,49168149,49454097,45478005,43585785,49472179,47257529,48775091,49211061,35809193,49031201,48434303,49306755,49098265,49573236,46367371,35508545,50293354,39824397,49637258,36526459&preCompiledProp=&excludePropIds=&addpropertyDataSet=',
 'https://www.magicbricks.com/mbsearch/propertySearch.html?propertyType_new=10002_10003_10021_10022,10001_10017,10000&postedSince=-1&city=3327&searchType=1&propertyType=10002,10003,10021,10022,10001,10017,10000&disWeb=Y&pType=10002,10003,10021,10022,10001,10017,10000&category=S&cusImgCount=0&groupstart=30&offset=0&maxOffset=431&attra

In [110]:
#Looping over the urls and scrpaing the property links
for url in urls:
    header={"user-agent":"Mozilla/4.0 (compatible; MSIE 6.0; Windows 98) XX"}
    soup=BeautifulSoup(requests.get(url,headers=header).text)
    for i in soup.find_all('div',class_="m-srp-card SRCard"):
        property_links.append(re.findall("https://www.magicbricks.com/.+",i.get("data-code"))[0].split(",")[0].strip("'"))
    if len(property_links) >500:
        break

In [111]:
len(property_links)

511

In [120]:
#saving links
links_data=pd.DataFrame(property_links,columns=["property_links"])
links_data.head()
links_data.to_csv("property_links.csv",index=False)

---------

<img src='product.png' />

In [2]:
links_data=pd.read_csv("property_links.csv")

In [3]:
#defining column widths
pd.set_option('display.max_colwidth',200)

In [16]:
links_data[:20]

,property_links
0,https://www.magicbricks.com/propertyDetails/3-BHK-2526-Sq-ft-Multistorey-Apartment-FOR-Sale-Hebbal-in-Bangalore&id=4d423339343735343735?sem=Y
1,https://www.magicbricks.com/propertyDetails/2-BHK-815-Sq-ft-Multistorey-Apartment-FOR-Sale-Kengeri-in-Bangalore&id=4d423436373635353737
2,https://www.magicbricks.com/propertyDetails/2-BHK-1376-Sq-ft-Multistorey-Apartment-FOR-Sale-Kadubeesanahalli-in-Bangalore&id=4d423439373537323134
3,https://www.magicbricks.com/propertyDetails/2-BHK-1141-Sq-ft-Multistorey-Apartment-FOR-Sale-Thambuchetty-Palya-in-Bangalore&id=4d423438343936333133
4,https://www.magicbricks.com/propertyDetails/2-BHK-970-Sq-ft-Multistorey-Apartment-FOR-Sale-Bellandur-in-Bangalore&id=4d423439333939323131
5,https://www.magicbricks.com/propertyDetails/3-BHK-1300-Sq-ft-Multistorey-Apartment-FOR-Sale-Electronic-City-Phase-2-in-Bangalore&id=4d423439313032353635
6,https://www.magicbricks.com/propertyDetails/3-BHK-1550-Sq-ft-Multistorey-Apartment-FOR-Sale-Kanakapura-Road-in-Bangalore&id=4d423439383330313538
7,https://www.magicbricks.com/propertyDetails/4-BHK-3335-Sq-ft-Multistorey-Apartment-FOR-Sale-Hebbal-in-Bangalore&id=4d423339343735363131
8,https://www.magicbricks.com/propertyDetails/2-BHK-880-Sq-ft-Multistorey-Apartment-FOR-Sale-Whitefield-in-Bangalore&id=4d423436393530313937
9,https://www.magicbricks.com/propertyDetails/2-BHK-883-Sq-ft-Multistorey-Apartment-FOR-Sale-Mysore-Road-in-Bangalore&id=4d423432393839303339?sem=Y


In [31]:
#Instantiating empty lists
price=[]
headline=[]
person=[]
Bedrooms=[]
Bathrooms=[]
Balcony=[]
Super_area=[]
Status=[]
Transaction_type=[]
Floor=[]
Car_parking=[]
Furnished_status=[]
desc=[]
Lift=[]
Price_Breakup=[]
Address=[]
Age_of_Construction=[]
Landmarks=[]

In [32]:
#using tqdm over for loop to get a progress bar!
from tqdm import tqdm

In [33]:
for url in tqdm(list(links_data.property_links)):
    header={"user-agent":"Mozilla/3.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36"}
    soup=BeautifulSoup(requests.get(url,headers=header).text)
    
    try:      #if by chance any link stops working price will get nil value and then that link wil be discarded
        price.append(re.sub("\n","",soup.find("div",class_="p_price").get_text()))
    except:
        price.append("nil")
    if price[-1]=="nil":
        price.pop()
        continue
        
    #headline
    headline.append(re.sub("\n","",soup.find("div",class_="propBhk").h1.span.get_text()))
    
    #person/builder/owner
    person.append(re.sub("\n","",soup.find("div",class_="nameValue").get_text()))

    #################################################################################################################
    p_title=[]
    pvalue=[]
    #scraping all the all the titles and their values from p_infoColumn block
    #then extracting required variabe values using title indexing
    for prop in soup.find_all("div",class_="p_infoColumn"):
        p_title.append(prop.find("div",class_="p_title").get_text().strip())
        pvalue.append(prop.find("div",class_="p_value").get_text().strip())

    #Bedrooms
    try:
        Bedrooms.append(re.search("\d+",pvalue[p_title.index("Bedrooms")]).group())
    except:
        Bedrooms.append("nil")

    #Bathrooms
    try:
        Bathrooms.append(pvalue[p_title.index("Bathrooms")])
    except:
        Bathrooms.append("nil")

    #Balcony
    try:
        Balcony.append(pvalue[p_title.index("Balcony")])
    except:
        try:
            Balcony.append(pvalue[p_title.index("Balconies")])
        except:
            Balcony.append("nil")

    #Super area
    try:
        Super_area.append(re.search("\d+",pvalue[p_title.index("Super area")]).group())
    except:
        Super_area.append("nil")

    #Status
    try:
        Status.append(pvalue[p_title.index("Status")])
    except:
        Status.append("nil")

    #Transaction type
    try:
        Transaction_type.append(pvalue[p_title.index("Transaction type")])
    except:
        Transaction_type.append("nil")

    #Floor
    try:
        Floor.append(pvalue[p_title.index("Floor")])
    except:
        Floor.append("nil")

    #Car parking
    try:
        Car_parking.append(pvalue[p_title.index("Car parking")])
    except:
        Car_parking.append("nil")

    #Furnished status
    try:
        Furnished_status.append(pvalue[p_title.index("Furnished status")])
    except:
        Furnished_status.append("nil")
    #################################################################################################################
    
    desc_block=soup.find("div",class_="descriptionCont")

    #description
    desc.append(desc_block.div.get_text())

    p_title=[]
    pvalue=[]
    #scraping all the all the titles and their values from p_infoRow block
    #then extracting required variabe values using title indexing
    for prop in desc_block.find_all("div",class_="p_infoRow"):
        try:
            pvalue.append(prop.find("div",class_="p_value").get_text().strip())
        except:
            pvalue.append("nil")
        try:
            p_title.append(prop.find("div",class_="p_title").get_text().strip())
        except:
            p_title.append("nil")


    #Lift
    try:
        Lift.append(pvalue[p_title.index("Lift")])
    except:
        Lift.append("nil")

    #Price Breakup
    try:
        Price_Breakup.append(pvalue[p_title.index("Price Breakup")])
    except:
        Price_Breakup.append("nil")

    #Address
    try:
        Address.append(pvalue[p_title.index("Address")])
    except:
        Address.append("nil")

    #Age of Construction
    try:
        Age_of_Construction.append(pvalue[p_title.index("Age of Construction")])
    except:
        Age_of_Construction.append("nil")

    #Landmarks
    try:
        Landmarks.append(pvalue[p_title.index("Landmarks")])
    except:
        Landmarks.append("nil")

    #     #var
    #     try:
    #         var.append(pvalue[p_title.index("var")])
    #     except:
    #         var.append("nil")


#     break
    

100%|████████████████████████████████████████████████████████████████████████████████| 511/511 [13:31<00:00,  1.59s/it]


_________

In [43]:
cols=[i.strip("\n") for i in """price=[]
headline=[]
person=[]
Bedrooms=[]
Bathrooms=[]
Super_area=[]
Status=[]
Transaction_type=[]
Floor=[]
Car_parking=[]
Furnished_status=[]
desc=[]
Lift=[]
Price_Breakup=[]
Address=[]
Age_of_Construction=[]
Landmarks=[]""".split("=[]")][:-1]
cols

['price',
 'headline',
 'person',
 'Bedrooms',
 'Bathrooms',
 'Super_area',
 'Status',
 'Transaction_type',
 'Floor',
 'Car_parking',
 'Furnished_status',
 'desc',
 'Lift',
 'Price_Breakup',
 'Address',
 'Age_of_Construction',
 'Landmarks']

In [112]:
#zipping all the variable lists and getting a dataframe out of it
data=list(zip(price,headline,person,Bedrooms,Bathrooms,Super_area,Status,Transaction_type,Floor,Car_parking,Furnished_status,desc,Lift,Price_Breakup,Address,Age_of_Construction,Landmarks))
frame=pd.DataFrame(data)
frame.columns=cols
frame

,price,headline,person,Bedrooms,Bathrooms,Super_area,Status,Transaction_type,Floor,Car_parking,Furnished_status,desc,Lift,Price_Breakup,Address,Age_of_Construction,Landmarks
0,₹2.57 Cr,3 BHK 2526 Sq-ft Flat,Gururaj,3,3,2526,Ready to Move,New Property,nil,1 Covered,Unfurnished,"\n\n\nSNN believes in Think BIG.Think ahead and of course we rely on our strong construction team who help us deliver our vision. When taking over SNN Clermont project, we wanted to ensure a large...",2,₹\n2.57 Cr,"Hebbal, Bangalore - North, Karnataka",nil,nil
1,₹45.0 Lac,2 BHK 815 Sq-ft Flat,Karthik,2,2,815,Under Construction,New Property,6 (Out of 6 Floors),nil,nil,"\n\n\nHill Crest is a luxurious and affordable residential apartment project by Sairam Dwellings located next to Vishweshwaraiah Layout 2nd Stage Bangalore OFF NICE ROAD, which is also close proxi...",2,"₹\n45.0 Lac\n\n\n₹\n2,000 \nMonthly","Kengeri, Bangalore - West, Karnataka",nil,nil
2,₹1.06 Cr,2 BHK 1376 Sq-ft Flat,Babu,2,2,1376,Ready to Move,New Property,nil,1 Covered,Unfurnished,"\n\n\nA Home Where Everything is a Blessing. A premium gated residential cluster of 232 luxury apartments spread over a pristinely manicured ground, Bhuvi by Amsha offers varying sizes of homes ac...",2,"₹\n1.06 Cr\n\n\n₹\n2,000 \nMonthly","Kadubeesanahalli, Bangalore - East, Karnataka",nil,nil
3,₹79.1 Lac,2 BHK 1141 Sq-ft Flat,Banupriya,2,2,1141,Under Construction,New Property,nil,None,nil,\n\n\nSun & Sanctum is a tall stepped architectural delight creating a unique skyline in the otherwise teeming city. The project is spread across 4 acres with 69% open space and a wide range of mo...,2,"₹\n79.1 Lac\n\n\n₹\n3,000 \nMonthly","Thambuchetty Palya, Bangalore - East, Karnataka",nil,nil
4,₹63.6 Lac,2 BHK 970 Sq-ft Flat,Nikhar Aventino,2,2,970,Under Construction,New Property,nil,1 Covered,nil,"\n\n\nLuxury takes on a new definition at Nikhar Aventino. Be it Picture-Perfect interiors, aesthetically designed structures, Innovative ideas in space management or the understated elegance of f...",2,₹\n63.6 Lac,"Bellandur, Bangalore - East, Karnataka",nil,nil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,₹45.0 Lac,2 BHKFlat,Kiran Kumar,2,2,1135,Ready to Move,Resale,3 (Out of 4 Floors),1 Covered,Unfurnished,\nMultistorey Apartment is located at the advantageous 3rd floor in a tower of total 4 floors. This spacious Multistorey Apartment was constructed approx. 10 to 15 years years back.\n,nil,₹\n45.0 Lac,"Bangalore, Karnataka",10 to 15 years,nil
504,₹1.20 Cr,"3 BHK Flat for sale in Sarjapur Road, Bangalore",Prakash prakash,3,2,1500,Ready to Move,Resale,7 (Out of 8 Floors),1 Covered,nil,\nThis property has 2 towers with all major aminites. ideal for IT professionals Also its fully furnished and ready to move in.\n,2,₹\n1.20 Cr,"Bellandur - Marathahalli Outer Ring Road, Off Sarjapur Road Bangalore, Karnataka, India., Sarjapur Road, Bangalore - East, Karnataka",New Construction,"next to banglare central,icici bank, cloud 9 , Cult ,reliance fresh"
505,₹43.2 Lac,"2400 Sq-ft Residential Plot/Land for Sale in Bescom City , Nandi hills , Bangalore",Afshan Riyaz Behelvi,nil,nil,nil,nil,Resale,nil,None,nil,"\n\n\nBescom City, situated in Nandi Hills, of Bangalore is one of the affordable housing options for those looking for a permanent settlement in such a majestic gateway to nature. Well, about for...",nil,"₹\n43.2 Lac\n\n\n₹\n 2,41,919\nApprox. Registration Charges\n\n\n\nSee Other Charges","Located @ BESCOM City, Devisettihalli, Karnataka 562103, Nandi hills, Bangalore - North, Delhi NCR",nil,Nandi Hills
506,₹1.65 Cr,7 BHK Owner Residential House,Shivaraj Patil,7,6,3500,Ready to Move,Resale,2 (Out of 3 Floors),4 Covered,Semi-Furnished,"\n\n\nAn amazing opportunity of owning a 7 BHK house is waiting for your attention!!! Up for sale, this abode located in Vinayaka layout-Medaralli, Bangalore is absolutely ready to move in with yo...",nil,₹\n1.65 Cr,"221., Vinayaka layout-Medaralli, Bangalore - North,

In [115]:
#getting the price in integer value
amount=[]
for i in frame.price.apply(lambda x:x.strip("₹")).apply(lambda x:x.lower()):
    x=i.split()[0]
    y=i.split()[1]
    if y=="cr":
        amount.append(int(float(x)*10000000))
    else:
        amount.append(int(float(x)*100000))
amount

[25700000,
 4500000,
 10600000,
 7909999,
 6360000,
 4550000,
 7040000,
 33600000,
 4990000,
 3750000,
 51600000,
 5050000,
 9260000,
 4100000,
 5330000,
 6180000,
 6520000,
 6740000,
 11399999,
 8420000,
 2680000,
 9000000,
 10800000,
 11399999,
 7820000,
 9900000,
 4470000,
 8850000,
 4500000,
 10600000,
 9510000,
 6970000,
 4490000,
 9980000,
 10200000,
 9500000,
 23000000,
 3960000,
 5000000,
 7790000,
 9400000,
 4560000,
 8900000,
 6390000,
 7980000,
 10300000,
 2810000,
 7730000,
 10800000,
 7230000,
 7820000,
 6640000,
 14600000,
 4510000,
 8040000,
 7950000,
 5390000,
 2400000,
 7980000,
 6750000,
 10100000,
 5850000,
 2750000,
 9200000,
 5970000,
 8550000,
 15600000,
 15400000,
 11900000,
 6150000,
 19500000,
 5580000,
 4490000,
 6990000,
 5140000,
 5910000,
 20000000,
 4680000,
 2790000,
 2039999,
 14100000,
 5200000,
 13799999,
 18600000,
 6900000,
 4950000,
 9430000,
 7159999,
 6980000,
 5950000,
 17500000,
 9740000,
 6730000,
 5430000,
 9600000,
 3150000,
 5300000,
 558000

In [116]:
#getting Price_per_sqft from price/Super_area
Price_per_sqft=[]
for i,j in zip(amount,frame.Super_area):
    if j=="nil":
        Price_per_sqft.append("nil")
    else:
        Price_per_sqft.append(int(i/int(j)))
Price_per_sqft

[10174,
 5521,
 7703,
 6932,
 6556,
 3500,
 4541,
 10074,
 5670,
 4246,
 9086,
 5163,
 5653,
 'nil',
 4602,
 5291,
 5596,
 4680,
 6386,
 6075,
 3098,
 5750,
 6118,
 5431,
 5750,
 6187,
 4608,
 6103,
 5590,
 7703,
 6659,
 6514,
 3499,
 5336,
 6219,
 7313,
 8354,
 5315,
 4739,
 5698,
 5696,
 4497,
 5597,
 4701,
 6487,
 5974,
 3098,
 5812,
 6118,
 5444,
 5750,
 6323,
 7185,
 4602,
 5864,
 5028,
 3500,
 'nil',
 6138,
 5075,
 6328,
 'nil',
 3205,
 6275,
 6251,
 7154,
 8000,
 5733,
 6367,
 4652,
 7121,
 5530,
 'nil',
 4943,
 4516,
 5372,
 7993,
 6902,
 2996,
 'nil',
 6080,
 3945,
 8214,
 6148,
 6634,
 'nil',
 7116,
 6509,
 4967,
 'nil',
 5298,
 5993,
 4908,
 4597,
 4247,
 3600,
 4453,
 5004,
 5221,
 6181,
 4603,
 5643,
 4332,
 4744,
 4763,
 4467,
 4194,
 8574,
 6033,
 4697,
 4517,
 4915,
 3299,
 3862,
 4173,
 4033,
 14545,
 7080,
 'nil',
 8997,
 5852,
 22483,
 9238,
 9000,
 32548,
 8274,
 6514,
 6800,
 15000,
 3287,
 'nil',
 6666,
 'nil',
 7131,
 3490,
 4851,
 7687,
 4827,
 7608,
 7262,
 563

In [117]:
frame["Price_per_sqft"]=Price_per_sqft
frame.price=amount

In [118]:
frame.to_excel("excel_output.xlsx",index=False)

In [120]:
frame.shape

(508, 18)

In [121]:
frame

,price,headline,person,Bedrooms,Bathrooms,Super_area,Status,Transaction_type,Floor,Car_parking,Furnished_status,desc,Lift,Price_Breakup,Address,Age_of_Construction,Landmarks,Price_per_sqft
0,25700000,3 BHK 2526 Sq-ft Flat,Gururaj,3,3,2526,Ready to Move,New Property,nil,1 Covered,Unfurnished,"\n\n\nSNN believes in Think BIG.Think ahead and of course we rely on our strong construction team who help us deliver our vision. When taking over SNN Clermont project, we wanted to ensure a large...",2,₹\n2.57 Cr,"Hebbal, Bangalore - North, Karnataka",nil,nil,10174
1,4500000,2 BHK 815 Sq-ft Flat,Karthik,2,2,815,Under Construction,New Property,6 (Out of 6 Floors),nil,nil,"\n\n\nHill Crest is a luxurious and affordable residential apartment project by Sairam Dwellings located next to Vishweshwaraiah Layout 2nd Stage Bangalore OFF NICE ROAD, which is also close proxi...",2,"₹\n45.0 Lac\n\n\n₹\n2,000 \nMonthly","Kengeri, Bangalore - West, Karnataka",nil,nil,5521
2,10600000,2 BHK 1376 Sq-ft Flat,Babu,2,2,1376,Ready to Move,New Property,nil,1 Covered,Unfurnished,"\n\n\nA Home Where Everything is a Blessing. A premium gated residential cluster of 232 luxury apartments spread over a pristinely manicured ground, Bhuvi by Amsha offers varying sizes of homes ac...",2,"₹\n1.06 Cr\n\n\n₹\n2,000 \nMonthly","Kadubeesanahalli, Bangalore - East, Karnataka",nil,nil,7703
3,7909999,2 BHK 1141 Sq-ft Flat,Banupriya,2,2,1141,Under Construction,New Property,nil,None,nil,\n\n\nSun & Sanctum is a tall stepped architectural delight creating a unique skyline in the otherwise teeming city. The project is spread across 4 acres with 69% open space and a wide range of mo...,2,"₹\n79.1 Lac\n\n\n₹\n3,000 \nMonthly","Thambuchetty Palya, Bangalore - East, Karnataka",nil,nil,6932
4,6360000,2 BHK 970 Sq-ft Flat,Nikhar Aventino,2,2,970,Under Construction,New Property,nil,1 Covered,nil,"\n\n\nLuxury takes on a new definition at Nikhar Aventino. Be it Picture-Perfect interiors, aesthetically designed structures, Innovative ideas in space management or the understated elegance of f...",2,₹\n63.6 Lac,"Bellandur, Bangalore - East, Karnataka",nil,nil,6556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,4500000,2 BHKFlat,Kiran Kumar,2,2,1135,Ready to Move,Resale,3 (Out of 4 Floors),1 Covered,Unfurnished,\nMultistorey Apartment is located at the advantageous 3rd floor in a tower of total 4 floors. This spacious Multistorey Apartment was constructed approx. 10 to 15 years years back.\n,nil,₹\n45.0 Lac,"Bangalore, Karnataka",10 to 15 years,nil,3964
504,12000000,"3 BHK Flat for sale in Sarjapur Road, Bangalore",Prakash prakash,3,2,1500,Ready to Move,Resale,7 (Out of 8 Floors),1 Covered,nil,\nThis property has 2 towers with all major aminites. ideal for IT professionals Also its fully furnished and ready to move in.\n,2,₹\n1.20 Cr,"Bellandur - Marathahalli Outer Ring Road, Off Sarjapur Road Bangalore, Karnataka, India., Sarjapur Road, Bangalore - East, Karnataka",New Construction,"next to banglare central,icici bank, cloud 9 , Cult ,reliance fresh",8000
505,4320000,"2400 Sq-ft Residential Plot/Land for Sale in Bescom City , Nandi hills , Bangalore",Afshan Riyaz Behelvi,nil,nil,nil,nil,Resale,nil,None,nil,"\n\n\nBescom City, situated in Nandi Hills, of Bangalore is one of the affordable housing options for those looking for a permanent settlement in such a majestic gateway to nature. Well, about for...",nil,"₹\n43.2 Lac\n\n\n₹\n 2,41,919\nApprox. Registration Charges\n\n\n\nSee Other Charges","Located @ BESCOM City, Devisettihalli, Karnataka 562103, Nandi hills, Bangalore - North, Delhi NCR",nil,Nandi Hills,nil
506,16500000,7 BHK Owner Residential House,Shivaraj Patil,7,6,3500,Ready to Move,Resale,2 (Out of 3 Floors),4 Covered,Semi-Furnished,"\n\n\nAn amazing opportunity of owning a 7 BHK house is waiting for your attention!!! Up for sale, this abode located in Vinayaka layout-Medaralli, Bangalore is absolutely ready to move in with yo...",nil,₹\n1.65 Cr,"22

---------

-------

### I tried finding Quicks facts in many pages but could'nt find any page with : 
* `Price comparison, Expected rent, Monthly EMI`
####  All other variables are sucessfully scraped: